In [8]:
import pandas as pd
from common import Data

In [9]:
df = pd.read_csv("2024_week18.csv")

In [10]:
def get_standings(df):
    df["season"] = "2024"
    result = Data.get_nwsl_standings(df)
    result["games"] = result["wins"] + result["draws"] + result["losses"]
    result["max_points"] = result["points"] + 3 * (26 - result["games"])
    return result[["team", "games", "points", "max_points"]].sort_values(["max_points"], ascending=False)


def add_results(df, scores):
    result = df.copy()

    for home, home_score, away, away_score in scores:
        result.loc[
            (df["home"] == home) & (df["away"] == away), ["home_score", "away_score"]
        ] = [home_score, away_score]

    return result


def get_matches_between(df, a, b):
    remaining = df[df["home_score"].isna()]
    count = len(
        remaining[
            ((remaining["home"] == a) & (remaining["away"] == b))
            | ((remaining["home"] == b) & (remaining["away"] == a))
        ].index
    )
    if count > 0:
        print(a, b, count)

In [11]:
# ("WAS", 0, "HOU", 0)
# ("NC", 0, "BFC", 0)
# ("POR", 0, "CHI", 0)
# ("LOU", 0, "LA", 0)
# ("ORL", 0, "KC", 0)
# ("RGN", 0, "NJY", 0)


new_schedule = add_results(
    df,
    [
        #("ORL", 0, "KC", 1),
        #("NC", 1, "BFC", 0),
        #("POR", 0, "CHI", 1),
    ],
)
get_standings(new_schedule)

,team,games,points,max_points
1,ORL,18,44,68
2,WAS,17,37,64
3,KC,18,35,59
4,NJY,18,34,58
5,NC,18,28,52
6,POR,18,27,51
7,CHI,18,23,47
8,BFC,18,21,45
9,LA,18,21,45
10,LOU,18,19,43


In [13]:
teams = ["NC", "POR", "CHI", "BFC", "LA"]

for a in teams:
    for b in teams:
        if a > b:
            get_matches_between(new_schedule, a, b)

NC CHI 1
NC BFC 2
NC LA 1
POR CHI 1
POR LA 2
